In this notebook Longitude and latitude columns are added in order for further visualisations and processing. This proccess take a lot of time, therefore it is done in separate notebook and columns added before all further manipulation with the dataset.

Import all necessary libraries

In [ ]:
import pandas as pd
import numpy as np
import time
import requests
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from tqdm import tqdm

Uploading dataset:

In [2]:
df = pd.read_parquet("../data/raw/pollution_dataset.parquet", engine="pyarrow")
df.head()

,Unnamed: 0,State Code,County Code,Site Num,Address,State,County,City,Date Local,NO2 Units,...,SO2 Units,SO2 Mean,SO2 1st Max Value,SO2 1st Max Hour,SO2 AQI,CO Units,CO Mean,CO 1st Max Value,CO 1st Max Hour,CO AQI
0,0,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-01,Parts per billion,...,Parts per billion,3.000000,9.0,21,13.0,Parts per million,1.145833,4.2,21,NaN
1,1,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-01,Parts per billion,...,Parts per billion,3.000000,9.0,21,13.0,Parts per million,0.878947,2.2,23,25.0
2,2,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-01,Parts per billion,...,Parts per billion,2.975000,6.6,23,NaN,Parts per million,1.145833,4.2,21,NaN
3,3,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-01,Parts per billion,...,Parts per billion,2.975000,6.6,23,NaN,Parts per million,0.878947,2.2,23,25.0
4,4,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-02,Parts per billion,...,Parts per billion,1.958333,3.0,22,4.0,Parts per million,0.850000,1.6,23,NaN


Set EPA credentials:

In [ ]:
AQS_EMAIL = "julia.shutko@gmail.com"
AQS_KEY = "bayhawk83"

EXTRACT UNIQUE SITE KEYS.

EPA data is uniquely defined by:
* State Code
* County Code
* Site Num

In [ ]:
site_keys = (
    df[["State Code", "County Code", "Site Num", "Address", "City", "State"]]
    .drop_duplicates()
    .reset_index(drop=True)
)

print("Unique EPA site keys:", len(site_keys))


CLEAN STREET ADDRESSES

EPA addresses are often messy. We standardize:

In [ ]:
def clean_address(row):
    addr = row["Address"]
    city = row["City"]
    state = row["State"]

    addr = str(addr).replace(",", " ").strip()
    city = str(city).replace(",", " ").strip()
    state = str(state).replace(",", " ").strip()

    # If address contains special chars or "[ ]", clean it
    addr = addr.replace("[", "").replace("]", "").replace("  ", " ")

    # Build full address
    full = f"{addr}, {city}, {state}, USA"

    return full

site_keys["CleanAddress"] = site_keys.apply(clean_address, axis=1)


AQS API — PRIMARY GEOCODING

EPA API is the official source for exact site coordinates.

In [ ]:
def get_coordinates_from_aqs(state, county, site, retries=3):
    url = "https://aqs.epa.gov/data/api/monitors/bySite"

    params = {
        "email": AQS_EMAIL,
        "key": AQS_KEY,
        "state": str(state).zfill(2),
        "county": str(county).zfill(3),
        "site": str(site).zfill(4)
    }

    for attempt in range(retries):
        try:
            r = requests.get(url, params=params, timeout=10)
            data = r.json()

            if "Data" in data and len(data["Data"]) > 0:
                d = data["Data"][0]
                return d.get("latitude"), d.get("longitude")
        except:
            time.sleep(1)

    return None, None


Initialize coordinate columns:

In [ ]:
site_keys["Latitude"] = np.nan
site_keys["Longitude"] = np.nan


Run EPA geocoding:

In [ ]:
for i, row in site_keys.iterrows():
    lat, lon = get_coordinates_from_aqs(
        row["State Code"], row["County Code"], row["Site Num"]
    )
    site_keys.at[i, "Latitude"] = lat
    site_keys.at[i, "Longitude"] = lon

    print(f"AQS {i+1}/{len(site_keys)} → {lat}, {lon}")
    time.sleep(0.2)  # avoid API rate limit


NOMINATIM FALLBACK FOR MISSING VALUES

Find which sites failed with AQS:

In [ ]:
missing = site_keys[site_keys["Latitude"].isna()]
print("Missing after AQS:", len(missing))

Configure Nominatim:

In [ ]:
geolocator = Nominatim(user_agent="epa_geocoder")
geo = RateLimiter(geolocator.geocode, min_delay_seconds=1)


Fallback function:

In [ ]:
def geocode_address_nominatim(address):
    try:
        loc = geo(address)
        if loc:
            return loc.latitude, loc.longitude
    except:
        return None, None
    return None, None


Run fallback only for missing sites:

In [ ]:
for i, row in missing.iterrows():
    lat, lon = geocode_address_nominatim(row["CleanAddress"])
    site_keys.at[i, "Latitude"] = lat
    site_keys.at[i, "Longitude"] = lon

    print(f"Nominatim {i+1}/{len(missing)} → {lat}, {lon}")


SAVE COORDINATE LOOKUP TABLE

In [ ]:
site_keys.to_parquet("../data/raw/epa_site_coordinates.parquet", index=False)
print("Saved → epa_site_coordinates.parquet")


VALIDATION OF RESULTS

In [ ]:
print("Remaining NaN coordinates:", site_keys[["Latitude", "Longitude"]].isna().sum())

failed_sites = site_keys[site_keys["Latitude"].isna()]
print(f"Unresolved sites: {len(failed_sites)}")
display(failed_sites.head(20))


MERGE COORDINATES BACK TO MAIN DATASET

In [ ]:
df_final = df.merge(
    site_keys[["State Code", "County Code", "Site Num", "Latitude", "Longitude"]],
    on=["State Code", "County Code", "Site Num"],
    how="left"
)
df_final.to_parquet("../data/processed/pollution_dataset_with_coordinates.parquet", index=False)
print("Saved → pollution_dataset_with_coordinates.parquet")